# PS02 - Regularization
DS325, Gettysburg College
Prof Eatai Roth

Due Monday Sep 22, 2025 5:00p

Total pts: 20

**Your Name**:
**Your Collaborators**:

## IMPORTANT INSTRUCTIONS:

 - When you submit your code, make sure that every cell runs without returning an error.
 - Once you have the results you need, edit out any extraneous code and outputs.
 - Do not rewrite code if it doesn't need to be rewritten. For example, in the sample analysis, I write all the import statements you should need. I also perform data cleaning and scaling. Do not redo those in your analysis.

### Problem 1 (and only)

In this assignment, you'll be doing a start-to-finish analysis of the King County Housing data. In class, we fit different linear regression models to the data, but never managed to capture the trend sufficiently. Your aim is to fix that.

You will add polynomial features in the hopes of better representing the shape of the data. You'll compare these results with the the original linear model.

### a. Fitting a LASSO model

Fit a Lasso model to the original data. Play around with values of alpha (the regularization strength) until you feel you have a good balance of generilization (fewer parameters) and goodness-of-fit ($R^2$).

 - Split the data 50-50 into training and testing set.
 - Fit your model and make predictions using both the training and test set.
 - Calculate the $R^2$ and RMSE for both training and test set. Compare these values.
    - Based on this comparison, how do you know if you are under- or over-fitting?
 - Iterate with different values of alpha to find a good model.
 - Report your $R^2$ and RMSE for your choice of model.
    - How many parameters does your model have (non-zero parameters)?
    - What are the 5 most important features according to the model (largest absolute coefficients)?
 - Plot the predicted vs actual prices as a scatter plot with the perfect-prediction line super-imposed.
    - Where does the model do well? Where does the model over- or under-estimate house values?

**Annotate and comment your code for readability PLEASE.**

 - I recommend writing as much of the code from scratch as possible. 
 - If copying from the notes, I recommend typing out the lines of code rather than copy-paste. 
 - If copy-pasting, I recommend reading every line of code and understanding what it does.


Below, I do some data cleaning for you. Start your work below the cell labeled 'Your work starts here'.

In [ ]:
import pandas as pd

pd.set_option('display.float_format', '{:.2f}'.format)
pd.set_option('display.max_colwidth', 15)

pd.options.display.max_colwidth = 15

import numpy as np
import matplotlib.pyplot as plt

In [49]:
housing_df = pd.read_csv('https://raw.githubusercontent.com/GettysburgDataScience/datasets/refs/heads/main/kc_house_data.csv')
housing_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21613 entries, 0 to 21612
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21613 non-null  int64  
 1   date           21613 non-null  object 
 2   price          21613 non-null  float64
 3   bedrooms       21613 non-null  int64  
 4   bathrooms      21613 non-null  float64
 5   sqft_living    21613 non-null  int64  
 6   sqft_lot       21613 non-null  int64  
 7   floors         21613 non-null  float64
 8   waterfront     21613 non-null  int64  
 9   view           21613 non-null  int64  
 10  condition      21613 non-null  int64  
 11  grade          21613 non-null  int64  
 12  sqft_above     21613 non-null  int64  
 13  sqft_basement  21613 non-null  int64  
 14  yr_built       21613 non-null  int64  
 15  yr_renovated   21613 non-null  int64  
 16  zipcode        21613 non-null  int64  
 17  lat            21613 non-null  float64
 18  long  

In [ ]:
housing_df.loc[housing_df['yr_renovated']==0, 'yr_renovated'] = housing_df.loc[housing_df['yr_renovated']==0, 'yr_built']

housing_df['yr_sold'] = housing_df['date'].apply(lambda d: int(d[0:4]))
housing_df['age_built'] = housing_df['yr_sold'] - housing_df['yr_built']
housing_df['age_reno'] = housing_df['yr_sold'] - housing_df['yr_renovated']

,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
count,21613.00,21613.00,21613.00,21613.00,21613.00,21613.00,21613.00,21613.00,21613.00,21613.00,21613.00,21613.00,21613.00,21613.00,21613.00,21613.00,21613.00,21613.00,21613.00,21613.00
mean,4580301520.86,540088.14,3.37,2.11,2079.90,15106.97,1.49,0.01,0.23,3.41,7.66,1788.39,291.51,1971.01,84.40,98077.94,47.56,-122.21,1986.55,12768.46
std,2876565571.31,367127.20,0.93,0.77,918.44,41420.51,0.54,0.09,0.77,0.65,1.18,828.09,442.58,29.37,401.68,53.51,0.14,0.14,685.39,27304.18
min,1000102.00,75000.00,0.00,0.00,290.00,520.00,1.00,0.00,0.00,1.00,1.00,290.00,0.00,1900.00,0.00,98001.00,47.16,-122.52,399.00,651.00
25%,2123049194.00,321950.00,3.00,1.75,1427.00,5040.00,1.00,0.00,0.00,3.00,7.00,1190.00,0.00,1951.00,0.00,98033.00,47.47,-122.33,1490.00,5100.00
50%,3904930410.00,450000.00,3.00,2.25,1910.00,7618.00,1.50,0.00,0.00,3.00,7.00,1560.00,0.00,1975.00,0.00,98065.00,47.57,-122.23,1840.00,7620.00
75%,7308900445.00,645000.00,4.00,2.50,2550.00,10688.00,2.00,0.00,0.00,4.00,8.00,2210.00,560.00,1997.00,0.00,98118.00,47.68,-122.12,2360.00,10083.00
max,9900000190.00,7700000.00,33.00,8.00,13540.00,1651359.00,3.50,1.00,4.00,5.00,13.00,9410.00,4820.00,2015.00,2015.00,98199.00,47.78,-121.31,6210.00,871200.00


In [57]:
columns_to_drop = ['id','date', 'zipcode', 'yr_built', 'yr_renovated']
housing_df.drop(columns = columns_to_drop, inplace=True)

#### Your work starts here

In [ ]:
'''your code here'''

**Questions**
 - *How many parameters does your model have?*
    - 'your response here'
 - *Which are your five most important parameters?*
    - 'your response here'
 - *What are your $R^2$ and RMSE for the training and testing data?*
    - 'your response here'
 - *Based on your model metrics, how do you know you are not over- or under-fitting?*
    - 'your response here'
 - *Based on your plots, where does the model do well? Where does the model over- or under-estimate house values?*
    - 'your response here'


### b. Fitting a Polynomial model

Do the same analysis as part a, but now using polynomial features.

 - Create polynomial features up to degree 2 with interaction terms.
    - How many features do you have after feature engineering?

Follow the same instructions as part a, you'll be asked the same questions at the end.

In [ ]:
'''your code here'''

**Questions**
 - *How many parameters does your model have?*
    - 'your response here'
 - *Which are your five most important parameters?*
    - 'your response here'
 - *What are your $R^2$ and RMSE for the training and testing data?*
    - 'your response here'
 - *Based on your model metrics, how do you know you are not over- or under-fitting?*
    - 'your response here'
 - *Based on your plots, where does the model do well? Where does the model over- or under-estimate house values?*
    - 'your response here'
 - *Did adding polynomial features improve the model? How so?*
    - 'your response here'


### Submission

To submit:
    - Uncomment the code cell below.
    - Run the code.

Submit as many times as you like.

In [ ]:
!git add *
!git commit -m "ps02 submission"
!git push origin main